In [79]:
import pandas as pd
import numpy as np

### Load Brands By Distributor

In [80]:
brands = 'data/BrandLinesByDistributors.xlsx'
brands_df = pd.read_excel(brands)

In [81]:
brands_df.head()

,58 Degrees & Holding,Unnamed: 1,7TH WEST,Unnamed: 3,ALIBI ALEWORKS,Unnamed: 5,ALTAMONT BEER WORKS,Unnamed: 7,ALVARADO STREET BREWING COMPANY,Unnamed: 9,...,WINE WAREHOUSE (CA16),Unnamed: 343,WINE WAREHOUSE (DO NOT BILL),Unnamed: 345,Woodhouse Blending & Brewing,Unnamed: 347,YOUNG'S MARKET COMPANY,Unnamed: 349,YOUNG'S MARKET COMPANY (DO NOT BILL),Unnamed: 351
0,Total Lines,8.0,Total Lines,9.0,Total Lines,24.0,Total Lines,413.0,Total Lines,125.0,...,Total Lines,153.0,Total Lines,39.0,Total Lines,17.0,Total Lines,78.0,Total Lines,6.0
1,Waterman Brewing El Guapo,1.0,Alameda Island Brewing,2.0,Alibi Berliner Reserve,1.0,Altamont Beer Works Maui Waui,69.0,Alvarado Street Mai Tai PA,25.0,...,Allagash White,8.0,North Coast Scrimshaw,14.0,Woodhouse Blending & Brewing,6.0,Line 39 Sauvignon Blanc,7.0,Nobilo Sauvignon Blanc,1.0
2,Amador Brewing Kolsch,1.0,Federation Behind You,1.0,Alibi Farmhouse Pineapple,1.0,Altamont Beer Works Hella Hoppy,51.0,Alvarado Street Monterey Beer,21.0,...,North Coast Scrimshaw,8.0,North Coast Scrimshaw Pilsner,3.0,Woodhouse Blending & Brewing Dark And Ominous,2.0,Line 39 Cabernet Sauvignon,6.0,Angelina Sonoma County Sauvignon Blanc,1.0
3,Matua Sauvignon Blanc,1.0,Del Cielo Brewing,1.0,Alibi Mr. Tartacular's Awkward Family Dinner,1.0,Altamont Beer Works Rich Mahogany,29.0,Alvarado Street Brewery Mai Tai,14.0,...,Lost Coast Tangerine Wheat,8.0,Anderson Valley Boont Amber,3.0,Wood House,2.0,Camelot Merlot,6.0,Angeline Wine Rose,1.0
4,Big Stump Brew Co Lions Crown,1.0,Alameda Island Brewing Hef-d,1.0,Alibi Rice Vice,1.0,Altamont Beer Works,25.0,Alvarado Street,10.0,...,North Coast Pranqster,6.0,Anderson Valley Boont Amber Ale,2.0,Woodhouse Blending Baby Kale Watermelon Cucumb...,1.0,Line 39 Chardonnay,6.0,Hess Chardonnay,1.0


### Remove Unnamed Columns

In [82]:
for col in brands_df.columns:
    if 'Unnamed' in col:
        del brands_df[col]

### Melt Columns

In [83]:
brands_df = pd.melt(brands_df)
brands_group = brands_df.groupby(['variable', 'value']).count().reset_index()

In [84]:
brands_group = brands_group[brands_group['value'] != 'Total Lines']

In [85]:
brands_group

,variable,value
0,58 Degrees & Holding,Amador Brewing Ipa
1,58 Degrees & Holding,Amador Brewing Kolsch
2,58 Degrees & Holding,Big Stump Brew Co Lions Crown
3,58 Degrees & Holding,Big Stump Brewing Rowan's Red Ale
4,58 Degrees & Holding,Matua Sauvignon Blanc
5,58 Degrees & Holding,Powder Keg Cabernet Sauvignon
7,58 Degrees & Holding,Two Rivers Ginger Cider
8,58 Degrees & Holding,Waterman Brewing El Guapo
9,7TH WEST,7 West Spray Can Pilsner
10,7TH WEST,Alameda Island Brewing


In [129]:
market_system_file = "data/MarketSystem.xlsx"
market_system = pd.read_excel(market_system_file)

In [133]:
market_system.head()

,distributor,serviceable,distactive,locid,city,address1,zip,tapname,bid,zonevalue,area,did,client,clientNumber,recurDescription,quantity
0,ALTAMONT BEER WORKS,True,True,1065193,Antioch,1 Marina Plz,94509,Altamont Beer Works Maui Waui,1054244,CA07,NaN,157,Smiths Landing Seafood Grill (Antioch/Marina),NaN,NaN,1
1,ALTAMONT BEER WORKS,True,True,1026931,Antioch,5019 Lone Tree Way,94531,Altamont Beer Works Hella Hoppy,3520,CA07,NaN,157,HD Burgers & More (Antioch/Lone Tree),NaN,NaN,1
2,ALTAMONT BEER WORKS,True,True,1026931,Antioch,5019 Lone Tree Way,94531,Altamont Beer Works Maui Waui,1054244,CA07,NaN,157,HD Burgers & More (Antioch/Lone Tree),NaN,NaN,1
3,ALTAMONT BEER WORKS,True,True,1044841,Brentwood,380 W Country Club Dr,94513,Altamont Beer Works Maui Waui,1054244,CA07,NaN,157,Mountain Mikes (Brentwood/W Country Club),NaN,NaN,1
4,ALTAMONT BEER WORKS,True,True,1955,Brentwood,6051 Lone Tree Way,94513,Altamont Beer Works Maui Waui,1054244,CA07,NaN,157,2M - Buffalo Wild Wings (Brentwood/Lone Tree),NaN,Clean on a 2 week cycle per gm,1


In [131]:
unique_locations = market_system.client.unique()

location_obj = {}
for x in unique_locations:
    loc = x
    df = market_system[market_system['client'] == loc]
    post = {'city': df.city.unique()[0],
           'distributors': df.distributor.unique(),
           'brands': df.tapname.unique()}
    location_obj[x] = post
location_obj

{'Smiths Landing Seafood Grill (Antioch/Marina)': {'city': 'Antioch',
  'distributors': array(['ALTAMONT BEER WORKS', 'BAY AREA BEVERAGE'], dtype=object),
  'brands': array(['Altamont Beer Works Maui Waui', 'Coors Light'], dtype=object)},
 'HD Burgers & More (Antioch/Lone Tree)': {'city': 'Antioch',
  'distributors': array(['ALTAMONT BEER WORKS', 'BAY AREA BEVERAGE'], dtype=object),
  'brands': array(['Altamont Beer Works Hella Hoppy', 'Altamont Beer Works Maui Waui',
         'Blue Moon Belgium White', 'Coors Light', "Lagunita's IPA"],
        dtype=object)},
 'Mountain Mikes (Brentwood/W Country Club)': {'city': 'Brentwood',
  'distributors': array(['ALTAMONT BEER WORKS'], dtype=object),
  'brands': array(['Altamont Beer Works Maui Waui'], dtype=object)},
 '2M - Buffalo Wild Wings (Brentwood/Lone Tree)': {'city': 'Brentwood',
  'distributors': array(['ALTAMONT BEER WORKS', 'BAY AREA BEVERAGE'], dtype=object),
  'brands': array(['Altamont Beer Works Maui Waui', 'Angel City Brewery IPA

In [132]:
market_trip_file = 'data/MarketTrip.xlsx'
market_trip = pd.read_excel(market_file)
market_trip.head()

,distributor,city,tapname,quantity,client,clientNumber
0,58 Degrees & Holding,Sacramento,Amador Brewing Ipa,1,58 Degrees & Holding Co. (Sacramento/18th),NaN
1,58 Degrees & Holding,Sacramento,Amador Brewing Kolsch,1,58 Degrees & Holding Co. (Sacramento/18th),NaN
2,58 Degrees & Holding,Sacramento,Big Stump Brew Co Lions Crown,1,58 Degrees & Holding Co. (Sacramento/18th),NaN
3,58 Degrees & Holding,Sacramento,Big Stump Brewing Rowan's Red Ale,1,58 Degrees & Holding Co. (Sacramento/18th),NaN
4,58 Degrees & Holding,Sacramento,Matua Sauvignon Blanc,1,58 Degrees & Holding Co. (Sacramento/18th),NaN


### DISTRIBUTOR CHECK

In [152]:
market_system_loc = market_system.client.unique()
i = 0

error_check = {'location': [],
               'distributor': [],
               'city': [],
               'tapname': [],
                'flag': [],
              'type': []}

for x in market_system_loc:
    loc = x
    df = market_trip[market_trip['client'] == loc]
    for index, row in df.iterrows():
        distributor = row['distributor']
        try:
            comparison_dist = location_obj[loc]['distributors']
            if distributor not in comparison_dist:
                error_check['location'].append(loc)
                error_check['distributor'].append(distributor)
                error_check['tapname'].append(row['tapname'])
                error_check['city'].append(row['city'])
                error_check['flag'].append(f'Not associated with {distributor} last cycle.')
                error_check['type'].append('dist')
        except:
            error_check['location'].append(loc)
            error_check['distributor'].append(distributor)
            error_check['tapname'].append(row['tapname'])
            error_check['city'].append(row['city'])
            error_check['flag'].append(f'Account was not cleaned last cycle.')
            error_check['type'].append('new acc, inc, alt cycle')
        
        tap = row['tapname']
        try:
            comparison_tap = location_obj[loc]['brands']
            if tap not in comparison_tap:
                error_check['location'].append(loc)
                error_check['distributor'].append(distributor)
                error_check['tapname'].append(row['tapname'])
                error_check['city'].append(row['city'])
                error_check['flag'].append(f'{tap} was not cleaned last cycle')
                error_check['type'].append(f'tap change')
        except:
            pass
        
loc_errors = pd.DataFrame(error_check)
loc_errors = loc_errors.groupby(['type', 'location', 'distributor', 'city', 'tapname', 'flag']).count().reset_index()
loc_errors

,type,location,distributor,city,tapname,flag
0,dist,2M - 99 Bottles Of Beer On The Wall (Santa Cru...,PREMIUM BEVERAGE,Santa Cruz,Dust Bowl,Not associated with PREMIUM BEVERAGE last cycle.
1,dist,2M - Burgers & Brew (Sacramento/R),BIKE DOG BREWING,Sacramento,Bike Dog Tarto,Not associated with BIKE DOG BREWING last cycle.
2,dist,2M - Depot (Sacramento/K),CROOKED LANE BREWING,Sacramento,Crooked Lane 1-2 Punch,Not associated with CROOKED LANE BREWING last ...
3,dist,2M - Hop Gardens (Sacramento/Franklin),GOLDEN BRANDS - SACRAMENTO,Sacramento,Boulevard Brewing Rye-On-Rye,Not associated with GOLDEN BRANDS - SACRAMENTO...
4,dist,2M - Hop Gardens (Sacramento/Franklin),MUSSETTER,Sacramento,Henhouse Super Duper Stoked,Not associated with MUSSETTER last cycle.
5,dist,2M - Mission St BBQ (Santa Cruz/Mission),Woodhouse Blending & Brewing,Santa Cruz,Wood House,Not associated with Woodhouse Blending & Brewi...
6,dist,2M - Mission St BBQ (Santa Cruz/Mission),Woodhouse Blending & Brewing,Santa Cruz,Woodhouse Blending & Brewing,Not associated with Woodhouse Blending & Brewi...
7,dist,2M - Stein's (Mountain View/Villa),CHRISSA IMPORTS,Mountain View,Moylan's Hop Craic IPA,Not associated with CHRISSA IMPORTS last cycle.
8,dist,2M - The Bank (Sacramento/J),BIKE DOG BREWING,Sacramento,Bike Dog Fog City,Not associated with BIKE DOG BREWING last cycle.
9,dist,2M - The Halford (Santa Clara/Halford),ALTAMONT BEER WORKS,Santa Clara,Altamont Beer Works,Not associated with ALTAMONT BEER WORKS last c...


In [120]:
location_obj['Whole Foods (Folsom/Palladio Pkwy)']['distributors']

array(['GOLDEN BRANDS - SACRAMENTO', 'MARKSTEIN BEVERAGE CO - SACTO'],
      dtype=object)

In [154]:
loc_errors.to_csv('loc_errors.csv')

In [153]:

for index, row in loc_errors.iterrows():
    if row['type'] == 'tap change':
        dist = row['distributor']
        df = brands_group[brands_group['variable'] == dist]
        brands = df.value.unique()
        if row['tapname'] not in brands:
            loc_errors.loc[index, 'brand_audit'] = "brand not associated w/ dist"
    else if row['type'] == 'dist':
        dist = row['distributor']